# 📊 Preparador de Lora de Hollowstrawberry

Este colab viene de [esta guía](https://huggingface.co/hollowstrawberry/stable-diffusion-guide/blob/main/README.md#index). Te permitirá obtener tus imágenes y tags para entrenar Loras.

Basado en el trabajo de [Kohya_ss y Linaqruf](https://colab.research.google.com/github/Linaqruf/kohya-trainer/blob/main/kohya-LoRA-dreambooth.ipynb#scrollTo=-Z4w3lfFKLjr). ¡Gracias!

| |GitHub|🇬🇧 English|🇪🇸 Spanish|
|:--|:-:|:-:|:-:|
| 📊 **Dataset Maker** | [![GitHub](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/github.svg)](https://github.com/hollowstrawberry/kohya-colab/blob/main/Dataset_Maker.ipynb) | [![Open in Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Dataset_Maker.ipynb) | [![Abrir en Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge-spanish.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Spanish_Dataset_Maker.ipynb) |
| ⭐ **Lora Trainer** | [![GitHub](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/github.svg)](https://github.com/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer.ipynb) | [![Open in Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer.ipynb) | [![Abrir en Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge-spanish.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Spanish_Lora_Trainer.ipynb) |

In [ ]:
#@title ## 🚩 Empezar aquí

import os
import shutil
import zipfile
from IPython.utils import capture
from google.colab import drive

#@markdown ### 1️⃣  Inicio
#@markdown Esta celda se conectará a tu Google Drive y creará las carpetas necesarias dentro de `lora_training`. <p>
#@markdown Tu nombre de proyecto será la carpeta donde trabajaremos. No se permiten espacios.
nombre_proyecto = "" #@param {type:"string"}
project_name = nombre_proyecto.strip()

if not project_name or any(c in project_name for c in " .()\"'\\"):
  print("Por favor elige un nombre válido.")
else:
  if not os.path.exists('/content/drive'):
    print("📂 Conectando a Google Drive...")
    drive.mount('/content/drive')

  #root_dir
  root_dir = "/content"
  deps_dir = os.path.join(root_dir,"deps")
  main_dir = os.path.join(root_dir,"drive/MyDrive/lora_training")
  datasets_dir = os.path.join(main_dir,"datasets")
  config_dir = os.path.join(main_dir,"config")
  images_folder = os.path.join(datasets_dir, project_name)
  config_folder = os.path.join(config_dir, project_name)

  for dir in [deps_dir, main_dir, datasets_dir, config_dir, images_folder, config_folder]:
    os.makedirs(dir, exist_ok=True)

  print(f"✅ ¡Proyecto {project_name} listo!")

In [ ]:
#@markdown ### 2️⃣ Obtener imágenes

import os
import html
import json
import time
from urllib.request import urlopen, Request
from IPython.display import Markdown, display
from google.colab import output as console

#@markdown Obtendremos imágenes de la galería de anime llamada [Gelbooru](https://gelbooru.com/). Las imágenes se organizan por miles de tags que describen todo acerca de una imagen. <p>
#@markdown * Si quieres encontrar y usar tus propias imágenes, ponlas dentro de la carpeta `lora_training/datasets/nombre_proyecto` en tu Google Drive.
#@markdown * Si quieres descargar capturas de episodios de anime, existe [este otro colab de otra persona](https://colab.research.google.com/drive/1oBSntB40BKzNmKceXUlkXzujzdQw-Ci7) aunque aquel es más complicado.

#@markdown Hasta 1000 imágenes se descargarán en un minuto, no debes abusar de ello. <p>
#@markdown Tus tags deben ser relevantes para lo que desees entrenar, y recomiendo incluir una puntuación mínima (score) y excluir la calificación explícita (puede hacer más difícil el entrenamiento).
#@markdown Las palabras van separadas por guionbajos, las tags van separadas por espacios, y usa - para excluir esa tag de tus resultados.
tags = "score:>10 -rating:explicit -loli -futanari -monochrome 1girl solo rem_(re:zero)" #@param {type:"string"}
##@markdown If an image is bigger than this resolution a smaller version will be downloaded instead.
max_resolution = 2048 #param {type:"slider", min:2048, max:8196, step:2048}
##@markdown Posts with a parent post are often minor variations of the same image.
include_posts_with_parent = True #param {type:"boolean"}

tags = tags.replace(" ", "+")\
           .replace("(", "%28")\
           .replace(")", "%29")\
           .replace(":", "%3a")\
          
url = "https://gelbooru.com/index.php?page=dapi&json=1&s=post&q=index&limit=100&tags={}".format(tags)
user_agent = "Mozilla/5.0 AppleWebKit/537.36 (KHTML, like Gecko; compatible; Googlebot/2.1; +http://www.google.com/bot.html) Chrome/93.0.4577.83 Safari/537.36"
limit = 100 # hardcoded by gelbooru
total_limit = 1000 # you can edit this if you want but I wouldn't recommend it
supported_types = (".png", ".jpg", ".jpeg")

def ubuntu_deps(url, name, dst):
  print("🏭 Instalando...")
  !wget -q --show-progress {url}
  if get_ipython().__dict__['user_ns']['_exit_code']:
    return
  with zipfile.ZipFile(name, 'r') as deps:
    deps.extractall(dst)
  !dpkg -i {dst}/*
  if get_ipython().__dict__['user_ns']['_exit_code']:
    return
  os.remove(name)
  shutil.rmtree(dst)
  console.clear()
  return True

if "step2_installed_flag" not in globals():
  if ubuntu_deps("https://huggingface.co/Linaqruf/fast-repo/resolve/main/deb-libs.zip", "deb-libs.zip", deps_dir):
    step2_installed_flag = True

def get_json(url):
  with urlopen(Request(url, headers={"User-Agent": user_agent})) as page:
    return json.load(page)

def filter_images(data):
  return [p["file_url"] if p["width"]*p["height"] <= max_resolution**2 else p["sample_url"]
          for p in data["post"]
          if (p["parent_id"] == 0 or include_posts_with_parent)
          and p["file_url"].lower().endswith(supported_types)]

def download_images():
  data = get_json(url)
  count = data["@attributes"]["count"]

  if count == 0:
    print("📷 No se encontraron resultados.")
    return

  print(f"🎯 Se encontraron {count} resultados")
  test_url = "https://gelbooru.com/index.php?page=post&s=list&tags={}".format(tags)
  display(Markdown(f"[¡Click aquí para verlos en tu navegador!]({test_url})"))
  inp = input("❓ Escribe \"si\" para continuar con la descarga: ")

  if inp.lower().strip() not in ("si", "sí"):
    print("❌ Download cancelled")
    return

  print("📩 Obteniendo lista de imágenes...")

  image_urls = set()
  image_urls = image_urls.union(filter_images(data))
  for i in range(total_limit // limit):
    count -= limit
    if count <= 0:
      break
    time.sleep(0.1)
    image_urls = image_urls.union(filter_images(get_json(url+f"&pid={i+1}")))

  scrape_file = os.path.join(config_folder, f"scraped_links.txt")
  with open(scrape_file, "w") as f:
    f.write("\n".join(image_urls))

  print(f"🌐 Enlaces guardados a {scrape_file}\n🔁 Empezando descarga en {images_folder} ...\n")
  old_img_count = len([f for f in os.listdir(images_folder) if f.lower().endswith(supported_types)])

  os.chdir(images_folder)
  !aria2c --console-log-level=error --summary-interval=10 -c -x 16 -k 1M -s 16 -i {scrape_file}

  new_img_count = len([f for f in os.listdir(images_folder) if f.lower().endswith(supported_types)])
  print(f"\n✅ Se han descargado {new_img_count - old_img_count} imágenes.")

download_images()


In [ ]:
#@markdown ### 3️⃣ Filtra tus imágenes
#@markdown Vamos a analizar y encontrar imágenes duplicadas usando la IA de FiftyOne, y las marcaremos con `eliminar`. <p>
#@markdown Cuando termine este proceso de varios minutos, aparecerá una zona interactiva bajo esta celda que te permitirá ver todas tus imágenes y marcar las que no te gusten con `eliminar` ttambién. <p>
#@markdown Si el área interactiva no carga después de un minuto, intenta activar las cookies o desactivar la protección del navegador para la página de Google Colab, ya que estos pueden interferir. <p>
#@markdown En cualquier caso, cuando estés satisfecho puedes mandar Enter en la casilla de texto arriba de la zona interactiva para finalizar y guardar los cambios.
#@markdown <p>&nbsp;<p>Qué tan similares deben ser dos imágenes para ser consideradas duplicadas. Recomiendo 0.97 a 0.99.
similarity_threshold = 0.985 #@param {type:"number"}

import os
from google.colab import output as console

model_name = "clip-vit-base32-torch"
supported_types = (".png", ".jpg", ".jpeg")
img_count = len(os.listdir(train_data_dir))
batch_size = min(250, img_count)

if "step3_installed_flag" not in globals():
  print("🏭 Instalando...\n")
  !pip install fiftyone ftfy
  if not get_ipython().__dict__['user_ns']['_exit_code']:
    console.clear()
    step3_installed_flag = True

import numpy as np
import fiftyone as fo
import fiftyone.zoo as foz
from fiftyone import ViewField as F
from sklearn.metrics.pairwise import cosine_similarity

non_images = [f for f in os.listdir(train_data_dir) if not f.lower().endswith(supported_types)]
if non_images:
  print(f"💥 Error: El archivo {non_images[0]} no es una imagen. Este programa no puede correr con otros archivos de por medio, lo lamento. Puedes usar los Extras para limpiar la carpeta.")
elif img_count == 0:
  print(f"💥 Error: No hay imágenes en {train_data_dir}")
else:
  print("💿 Analizando imágenes...")
  dataset = fo.Dataset.from_dir(train_data_dir, dataset_type=fo.types.ImageDirectory)
  model = foz.load_zoo_model(model_name)
  embeddings = dataset.compute_embeddings(model, batch_size=batch_size)

  batch_embeddings = np.array_split(embeddings, batch_size)
  similarity_matrices = []
  max_size_x = max(array.shape[0] for array in batch_embeddings)
  max_size_y = max(array.shape[1] for array in batch_embeddings)

  for i, batch_embedding in enumerate(batch_embeddings):
      similarity = cosine_similarity(batch_embedding)
      #Pad 0 for np.concatenate
      padded_array = np.zeros((max_size_x, max_size_y))
      padded_array[0:similarity.shape[0], 0:similarity.shape[1]] = similarity
      similarity_matrices.append(padded_array)

  similarity_matrix = np.concatenate(similarity_matrices, axis=0)
  similarity_matrix = similarity_matrix[0:embeddings.shape[0], 0:embeddings.shape[0]]

  similarity_matrix = cosine_similarity(embeddings)
  similarity_matrix -= np.identity(len(similarity_matrix))

  dataset.match(F("max_similarity") > similarity_threshold)
  dataset.tags = ["eliminar", "tiene_duplicados"]

  id_map = [s.id for s in dataset.select_fields(["id"])]
  samples_to_remove = set()
  samples_to_keep = set()

  for idx, sample in enumerate(dataset):
      if sample.id not in samples_to_remove:
          # Keep the first instance of two duplicates
          samples_to_keep.add(sample.id)
          
          dup_idxs = np.where(similarity_matrix[idx] > similarity_threshold)[0]
          for dup in dup_idxs:
              # We kept the first instance so remove all other duplicates
              samples_to_remove.add(id_map[dup])

          if len(dup_idxs) > 0:
              sample.tags.append("tiene_duplicados")
              sample.save()
      else:
          sample.tags.append("eliminar")
          sample.save()

  console.clear()

  sidebar_groups = fo.DatasetAppConfig.default_sidebar_groups(dataset)
  for group in sidebar_groups[2:]:
    group.expanded = False
  dataset.app_config.sidebar_groups = sidebar_groups
  dataset.save()
  session = fo.launch_app(dataset)

  print("❗ Espera un minuto mientras carga la zona interactiva. Si no carga lee arriba.")
  print("❗ Cuando esté listo verás una cuadrícula con tus imágenes.")
  print("❗ Al lado izquierdo selecciona la etiqueta \"eliminar\" para ver cuáles imágenes serán borradas.")
  print("❗ Puedes marcar tus propias imágenes no deseadas con \"eliminar\" al seleccionarlas y luego apretar el ícono de etiqueta en la parte de arriba.")
  input("⭕ Cuando termines, manda Enter aquí para guardar los cambios: ")

  session.refresh()
  fo.close_app()
  console.clear()
  print("💾 Guardando...")

  kys = [s for s in dataset if "eliminar" in s.tags]
  dataset.remove_samples(kys)
  real_project_name = project_name if "/" not in project_name else project_name[project_name.rfind("/")+1:]
  previous_folder = images_folder[:images_folder.rfind("/")]
  dataset.export(export_dir=os.path.join(images_folder, real_project_name), dataset_type=fo.types.ImageDirectory)
  
  temp_suffix = "_temp"
  !mv {images_folder} {images_folder}{temp_suffix}
  !mv {images_folder}{temp_suffix}/{real_project_name} {images_folder}
  !rm -r {images_folder}{temp_suffix}

  print(f"✅ Se han eliminado {len(kys)} imágenes no deseadas, quedan {len(os.listdir(train_data_dir))} imágenes.")


In [ ]:
#@markdown ### 4️⃣ Descripciones de imágenes
#@markdown Usaremos inteligencia artificial para describir tus imágenes, específicamente [Waifu Diffusion](https://huggingface.co/SmilingWolf/wd-v1-4-swinv2-tagger-v2) en el caso de anime (etiquetas/tags) y [BLIP](https://huggingface.co/spaces/Salesforce/BLIP) en el caso de fotorealismo (subtítulos/captions).
#@markdown Estas descripciones que van junto a tus imágenes mejoran notablemente la calidad de tu Lora a la hora de entrenar. <p>
metodo = "Anime" #@param ["Anime", "Fotorealismo"]
method = metodo
#@markdown **Anime:** El umbral es el nivel de certeza al que debe llegar la IA para asignar cada tag. Menor umbral = Más tags.
umbral = 0.35 #@param {type:"slider", min:0.0, max:1.0, step:0.01}
tag_threshold = umbral
#@markdown **Fotorealismo:** El mínmimo y máximo largo de cada subtítulo (medido en tokens/palabras).
largo_minimo = 10 #@param {type:"number"}
caption_min = largo_minimo
largo_maximo = 75 #@param {type:"number"}
caption_max = largo_maximo

%env PYTHONPATH=/env/python
import os
from google.colab import output as console
from IPython import get_ipython

os.chdir("/content")
kohya = "/content/kohya-trainer"
if not os.path.exists(kohya):
  !git clone https://github.com/Linaqruf/kohya-trainer {kohya}
  os.chdir(kohya)
  !git reset --hard 86de685a8c37e60a610d08cbece3da6b3a553bc0
  os.chdir("/content")

if "Anime" in method:
  if "step4a_installed_flag" not in globals():
    print("🏭 Instalando...\n")
    !pip install tensorflow==2.10.1 huggingface-hub==0.12.0 accelerate==0.15.0 transformers==4.26.0 diffusers[torch]==0.10.2 einops==0.6.0 safetensors==0.2.6
    if not get_ipython().__dict__['user_ns']['_exit_code']:
      console.clear()
      step4a_installed_flag = True

  print("🚶‍♂️ Iniciando programa...\n")

  %env PYTHONPATH={kohya}
  !python {kohya}/finetune/tag_images_by_wd14_tagger.py \
    {images_folder} \
    --repo_id=SmilingWolf/wd-v1-4-swinv2-tagger-v2 \
    --model_dir=/content \
    --thresh={tag_threshold} \
    --batch_size=8 \
    --caption_extension=.txt \
    --force_download

  if not get_ipython().__dict__['user_ns']['_exit_code']:
    print("removing underscores...")
    from collections import Counter
    top_tags = Counter()
    for txt in [f for f in os.listdir(images_folder) if f.lower().endswith(".txt")]:
      with open(os.path.join(images_folder, txt), 'r') as f:
        tags = [t.strip() for t in f.read().split(",")]
        tags = [t.replace("_", " ") if len(t) > 3 else t for t in tags]
      top_tags.update(tags)
      with open(os.path.join(images_folder, txt), 'w') as f:
        f.write(", ".join(tags))

    %env PYTHONPATH=/env/python
    console.clear()
    print(f"📊 Finalizado. Aquí están las 50 tags más comunes en tus imágenes:")
    print("\n".join(f"{k} ({v})" for k, v in top_tags.most_common(50)))
    
else: # Photorealism
  if "step4b_installed_flag" not in globals():
    print("🏭 Instalando...\n")
    !pip install timm==0.6.12 fairscale==0.4.13 transformers==4.26.0 requests==2.28.2 accelerate==0.15.0 diffusers[torch]==0.10.2 einops==0.6.0 safetensors==0.2.6
    if not get_ipython().__dict__['user_ns']['_exit_code']:
      console.clear()
      step4b_installed_flag = True

  print("🚶‍♂️ Iniciando programa...\n")

  os.chdir(kohya)
  %env PYTHONPATH={kohya}
  !python {kohya}/finetune/make_captions.py \
    {images_folder} \
    --beam_search \
    --max_data_loader_n_workers=2 \
    --batch_size=8 \
    --min_length={caption_min} \
    --max_length={caption_max} \
    --caption_extension=.txt

  if not get_ipython().__dict__['user_ns']['_exit_code']:
    import random
    captions = [f for f in os.listdir(images_folder) if f.lower().endswith(".txt")]
    sample = []
    for txt in random.sample(captions, min(10, len(captions))):
      with open(os.path.join(images_folder, txt), 'r') as f:
        sample.append(f.read())

    os.chdir("/content")
    %env PYTHONPATH=/env/python
    console.clear()
    print(f"📊 Finalizado. Aquí hay {len(sample)} ejemplos de subtítulos en tus imágenes:")
    print("".join(sample))



In [ ]:
import os
#@markdown ### 5️⃣ Filtra tus tags
#@markdown Puedes hacer modificaciones en las tags de tus imágenes cuantas veces quieras. <p>

#@markdown Añadir una palabra de activación a tu Lora, útil para mejorar el entrenamiento y usarlo en tus prompts. En el entrenamiento debes poner `keep_tokens` igual a 1.<p>
#@markdown Si quitas tags comunes como el color de pelo/ojos éstas serán "absorbidas" por tu palabra de activación.
palabra_de_activacion = "" #@param {type:"string"}
global_activation_tag = palabra_de_activacion
quitar_tags = "virtual youtuber" #@param {type:"string"}
remove_tags = quitar_tags
#@markdown <p>&nbsp;<p> En esta zona avanzada puedes realizar reemplazos o combinaciones de tags para así mejorar su calidad. Puedes reemplazar 1 tag por varias, o varias por 1, o una por otra, etc. También puedes añadir palabras de activación específicas.
buscar_tags = "" #@param {type:"string"}
search_tags = buscar_tags
reemplazar_con = "" #@param {type:"string"}
replace_with = reemplazar_con
modo_de_busqueda = "OR (puede tener cualquiera)" #@param ["OR (puede tener cualquiera)", "AND (debe tener todo)"]
search_mode = modo_de_busqueda
tag_nueva_se_convierte_en_palabra_de_activacion = False #@param {type:"boolean"}
new_becomes_activation_tag = tag_nueva_se_convierte_en_palabra_de_activacion
#@markdown Estas pueden ser útiles a veces. Ten cuidado, pueden quitar las palabras de activación previas.
ordenar_alfabeticamente = False #@param {type:"boolean"}
sort_alphabetically = ordenar_alfabeticamente
quitar_duplicados = False #@param {type:"boolean"}
remove_duplicates = quitar_duplicados

def split_tags(tagstr):
  return [s.strip() for s in tagstr.split(",") if s.strip()]

activation_tag_list = split_tags(global_activation_tag)
remove_tags_list = split_tags(remove_tags)
search_tags_list = split_tags(search_tags)
replace_with_list = split_tags(replace_with)
replace_new_list = [t for t in replace_with_list if t not in search_tags_list]

replace_with_list = [t for t in replace_with_list if t not in replace_new_list]
replace_new_list.reverse()
activation_tag_list.reverse()

remove_count = 0
replace_count = 0

for txt in [f for f in os.listdir(images_folder) if f.lower().endswith(".txt")]:

  with open(os.path.join(images_folder, txt), 'r') as f:
    tags = [s.strip() for s in f.read().split(",")]

  if remove_duplicates:
    tags = list(set(tags))
  if sort_alphabetically:
    tags.sort()

  for rem in remove_tags_list:
    if rem in tags:
      remove_count += 1
      tags.remove(rem)

  if "AND" in search_mode and all(r in tags for r in search_tags_list) \
      or "OR" in search_mode and any(r in tags for r in search_tags_list):
    replace_count += 1
    for rem in search_tags_list:
      if rem in tags:
        tags.remove(rem)
    for add in replace_with_list:
      if add not in tags:
        tags.append(add)
    for new in replace_new_list:
      if new_becomes_activation_tag:
        if new in tags:
          tags.remove(new)
        tags.insert(0, new)
      else:
        if new not in tags:
          tags.append(new)

  for act in activation_tag_list:
    if act in tags:
      tags.remove(act)
    tags.insert(0, act)

  with open(os.path.join(images_folder, txt), 'w') as f:
    f.write(", ".join(tags))

if remove_tags:
  print(f"\n🚮 Se han quitado {remove_count} tags.")
if search_tags:
  print(f"\n💫 Se han hecho {replace_count} reemplazos.")
print("\n✅ ¡Listo!")


In [ ]:
#@markdown ### 6️⃣  Listo
#@markdown Ahora debes estar listo para [entrenar tu Lora](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Spanish_Lora_Trainer.ipynb).

from IPython.display import Markdown, display
display(Markdown(f"🦀 [Click aquí para abrir el colab de entrenamiento](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Spanish_Lora_Trainer.ipynb) "))


## *️⃣ Extras

In [ ]:
#@markdown ### 📈 Analizar tags
#@markdown Volver a ver las tags más comunes en tus imágenes.
ver_top = 200 #@param {type:"number"}
show_top_tags = ver_top
from collections import Counter
top_tags = Counter()

for txt in [f for f in os.listdir(train_data_dir) if f.lower().endswith(".txt")]:
  with open(os.path.join(train_data_dir, txt), 'r') as f:
    top_tags.update([s.strip() for s in f.read().split(",")])

top_tags = Counter(top_tags)
print(f"📊 Tus {show_top_tags} tags más comunes:")
for k, v in top_tags.most_common(show_top_tags):
  print(f"{k} ({v})")

In [ ]:
#@markdown ### 📂 Extraer datos
#@markdown Es lento subir muchos archivos pequeños, si quieres puedes subir un zip y extraerlo aquí.
zip = "/content/drive/MyDrive/lora_training/datasets/warrior.zip" #@param {type:"string"}
extract_to = "/content/drive/MyDrive/lora_training/datasets/" #@param {type:"string"}

import os, zipfile

if not os.path.exists('/content/drive'):
  from google.colab import drive
  print("📂 Connecting to Google Drive...")
  drive.mount('/content/drive')

with zipfile.ZipFile(zip, 'r') as f:
  f.extractall(extract_to)

print("✅ Done")


In [ ]:
#@markdown ### 🔢 Contar archivos
#@markdown Google Drive hace imposible contar los archivos en una carpeta, por lo que aquí puedes ver la cantidad de archivos en carpetas y subcarpetas.
carpeta = "/content/drive/MyDrive/lora_training/datasets" #@param {type:"string"}
folder = carpeta

import os
from google.colab import drive

if not os.path.exists('/content/drive'):
    print("📂 Conectando a Google Drive...\n")
    drive.mount('/content/drive')

tree = {}
for i, (root, dirs, files) in enumerate(os.walk(folder)):
  images = len([f for f in files if f.lower().endswith((".png", ".jpg", ".jpeg"))])
  captions = len([f for f in files if f.lower().endswith(".txt")])
  others = len(files) - images - captions
  path = root[folder.rfind("/")+1:]
  tree[path] = None if not images and not captions and not others \
                    else f"{images:>4} images | {captions:>4} text files | {others:>4} other files"
pad = max(len(k) for k in tree)
print("\n".join(f"📁{k.ljust(pad)} | {v}" for k, v in tree.items() if v))


In [ ]:
#@markdown ### 🚮 Limpiar carpeta
#@markdown Cuidado, borra todos los archivos que no sean imágenes de la carpeta del proyecto.

!find {images_folder} -type f ! \( -name '*.png' -o -name '*.jpg' -o -name '*.jpeg' \) -delete